In [10]:
import pandas as pd
import csv

csv.field_size_limit(100000000)

in_file = "./web-crawler/kowiki/kowiki_20211014.csv"
out_file = "./web-crawler/kowiki/kowiki.txt"
SEPARATOR = u"\u241D"
df = pd.read_csv(in_file, sep=SEPARATOR, engine="python", encoding='UTF-8')
with open(out_file, "w", encoding='UTF-8') as f:
  for index, row in df.iterrows():
    f.write(row["text"]) # title 과 text를 중복 되므로 text만 저장 함
    f.write("\n\n\n\n") # 구분자

In [12]:
import sentencepiece as spm

corpus = "./web-crawler/kowiki/kowiki.txt"
prefix = "./web-crawler/kowiki/kowiki"
vocab_size = 8000
spm.SentencePieceTrainer.train(
    f"--input={corpus} --model_prefix={prefix} --vocab_size={vocab_size + 7}" + 
    " --model_type=bpe" +
    " --max_sentence_length=999999" + # 문장 최대 길이
    " --pad_id=0 --pad_piece=[PAD]" + # pad (0)
    " --unk_id=1 --unk_piece=[UNK]" + # unknown (1)
    " --bos_id=2 --bos_piece=[BOS]" + # begin of sequence (2)
    " --eos_id=3 --eos_piece=[EOS]" + # end of sequence (3)
    " --user_defined_symbols=[SEP],[CLS],[MASK]") # 사용자 정의 토큰

In [16]:
vocab_file = "./web-crawler/kowiki/kowiki.model"
vocab = spm.SentencePieceProcessor()
vocab.load(vocab_file)

lines = [
  "겨울이 되어서 날씨가 무척 추워요.",
  "이번 성탄절은 화이트 크리스마스가 될까요?",
  "겨울에 감기 조심하시고 행복한 연말 되세요."
]

for line in lines:
    pieces = vocab.encode_as_pieces(line)
    ids = vocab.encode_as_ids(line)
    print(line)
    print(pieces)
    print(ids)
    print()

겨울이 되어서 날씨가 무척 추워요.
['▁겨울', '이', '▁되어', '서', '▁날', '씨', '가', '▁무', '척', '▁추', '워', '요', '.']
[3189, 3536, 629, 3550, 696, 3975, 3547, 107, 4146, 198, 3904, 3708, 3538]

이번 성탄절은 화이트 크리스마스가 될까요?
['▁이번', '▁성', '탄', '절', '은', '▁화', '이트', '▁크리스', '마', '스가', '▁될', '까', '요', '?']
[2918, 88, 3916, 3871, 3552, 268, 661, 1909, 3612, 761, 1427, 3747, 3708, 4208]

겨울에 감기 조심하시고 행복한 연말 되세요.
['▁겨울', '에', '▁감', '기', '▁조', '심', '하', '시', '고', '▁행', '복', '한', '▁연', '말', '▁되', '세', '요', '.']
[3189, 3539, 201, 3554, 53, 3780, 3544, 3562, 3548, 240, 3820, 3551, 62, 3774, 452, 3631, 3708, 3538]



In [17]:
import json

def prepare_train(vocab, infile, outfile):
    df = pd.read_csv(infile, sep="\t", engine="python")
    with open(outfile, "w") as f:
        for index, row in df.iterrows():
            document = row["document"]
            if type(document) != str:
                continue
            instance = { "id": row["id"], "doc": vocab.encode_as_pieces(document), "label": row["label"] }
            f.write(json.dumps(instance))
            f.write("\n")

In [18]:
prepare_train(vocab, "./web-crawler/NaverSent/ratings_train.txt", "./web-crawler/NaverSent/ratings_train.json")
prepare_train(vocab, "./web-crawler/NaverSent/ratings_test.txt", "./web-crawler/NaverSent/ratings_test.json")